In [2]:
from decimal import Decimal

stat_table_items = [{'Date': '20240507', 'OdsCode': 'H81109', 'StatisticID': 'ApplicationData#d26b65af-530b-4d8e-a919-4999650f5d48', 'ActiveUserIdsHashed': ['4873620d0b476b13ee571a28cc315870', 'fake_user_id_hashed', '2nd_fake_user_id_hashed']}, {'Date': '20240509', 'OdsCode': 'H81109', 'StatisticID': 'ApplicationData#d26b65af-530b-4d8e-a919-4999650f5d48', 'ActiveUserIdsHashed': ['4873620d0b476b13ee571a28cc315870', 'ba81803adac3c816b6cbaf67bf33022a']}, {'Date': '20240509', 'OdsCode': 'M85143', 'StatisticID': 'ApplicationData#f2ef5173-485b-45e2-ba7d-b5bf6594c4f0', 'ActiveUserIdsHashed': ['6f1af742e351ce63d8ed275d4bec8d8f']}, {'Date': '20240509', 'DailyCountStored': Decimal('0'), 'NumberOfPatients': Decimal('4'), 'AverageRecordsPerPatient': Decimal('4.5'), 'DailyCountDownloaded': '4', 'DailyCountViewed': '35', 'OdsCode': 'M85143', 'DailyCountDeleted': '1', 'StatisticID': 'OrganisationData#6301dea1-996a-4f9b-98b8-a05cec4835d7'}, {'Date': '20240509', 'DailyCountStored': Decimal('0'), 'NumberOfPatients': Decimal('9'), 'AverageRecordsPerPatient': Decimal('2.777777777777777777777777778'), 'DailyCountDownloaded': '1', 'DailyCountViewed': '15', 'OdsCode': 'H81109', 'DailyCountDeleted': '1', 'StatisticID': 'OrganisationData#6d5a4847-4e9a-48d8-93c3-68bede2388a0'}, {'TotalSizeOfRecordsInMegabytes': Decimal('2.19059467315673828125'), 'Date': '20240509', 'TotalNumberOfRecords': Decimal('25'), 'NumberOfDocumentTypes': Decimal('1'), 'OdsCode': 'H81109', 'StatisticID': 'RecordStoreData#64f00ad8-8d11-4721-9111-48ec3ecf8ce8'}, {'TotalSizeOfRecordsInMegabytes': Decimal('1.7578678131103515625'), 'Date': '20240509', 'TotalNumberOfRecords': Decimal('18'), 'NumberOfDocumentTypes': Decimal('1'), 'OdsCode': 'M85143', 'StatisticID': 'RecordStoreData#a06eaca2-8a5d-4ffc-ad0b-72b48c3dae82'}, {'TotalSizeOfRecordsInMegabytes': Decimal('0.335521697998046875'), 'Date': '20240509', 'TotalNumberOfRecords': Decimal('0'), 'NumberOfDocumentTypes': Decimal('1'), 'OdsCode': 'ODS_CODE_UNKNOWN', 'StatisticID': 'RecordStoreData#a74412b5-0700-4d18-96ca-809b646fbf8e'}, {'Date': '20240508', 'DailyCountStored': Decimal('5'), 'NumberOfPatients': Decimal('5'), 'AverageRecordsPerPatient': Decimal('3'), 'DailyCountDownloaded': '5', 'DailyCountViewed': '5', 'DailyCountDeleted': '5', 'OdsCode': 'H81109', 'StatisticID': 'OrganisationData#320666AD-30C7-4853-AAA1-91EC0E88924C'}, {'Date': '20240513', 'OdsCode': 'H81109', 'StatisticID': 'ApplicationData#48937633-8422-4bf5-bb57-f30fca021f3b', 'ActiveUserIdsHashed': ['6f1af742e351ce63d8ed275d4bec8d8f']}, {'Date': '20240513', 'OdsCode': 'M85143', 'StatisticID': 'ApplicationData#7e2f23cc-8922-42dc-83fc-faf86bcdf771', 'ActiveUserIdsHashed': ['6f1af742e351ce63d8ed275d4bec8d8f']}, {'Date': '20240513', 'DailyCountStored': Decimal('0'), 'NumberOfPatients': Decimal('9'), 'AverageRecordsPerPatient': Decimal('2.777777777777777777777777778'), 'DailyCountDownloaded': Decimal('0'), 'DailyCountViewed': Decimal('0'), 'OdsCode': 'H81109', 'DailyCountDeleted': Decimal('0'), 'StatisticID': 'OrganisationData#cdc7cef8-16c3-4cbc-b4b1-b47756f29847'}, {'Date': '20240513', 'DailyCountStored': Decimal('0'), 'NumberOfPatients': Decimal('4'), 'AverageRecordsPerPatient': Decimal('4.5'), 'DailyCountDownloaded': Decimal('0'), 'DailyCountViewed': Decimal('0'), 'OdsCode': 'M85143', 'DailyCountDeleted': Decimal('0'), 'StatisticID': 'OrganisationData#ede69cb8-ad8c-405f-abe4-e69c80db6348'}, {'TotalSizeOfRecordsInMegabytes': Decimal('0.88645648956298828125'), 'Date': '20240513', 'TotalNumberOfRecords': Decimal('18'), 'NumberOfDocumentTypes': Decimal('1'), 'OdsCode': 'M85143', 'StatisticID': 'RecordStoreData#675241cd-3615-4459-a927-fc0b4f1c8547'}, {'TotalSizeOfRecordsInMegabytes': Decimal('1.41417980194091796875'), 'Date': '20240513', 'TotalNumberOfRecords': Decimal('25'), 'NumberOfDocumentTypes': Decimal('1'), 'OdsCode': 'H81109', 'StatisticID': 'RecordStoreData#eaa28be5-cea0-40de-b668-449f8f1137d5'}]


In [73]:
from models.statistics import (
    load_from_dynamodb_items,
)
import polars as pl
import polars.selectors as cs

loaded_data = load_from_dynamodb_items(stat_table_items)

df = pl.DataFrame(loaded_data.record_store_data).with_columns(cs.by_dtype(pl.datatypes.Decimal).cast(pl.Float64))

get_most_recent_record = pl.all().sort_by('date').last()
summarised_data = df.group_by('ods_code').agg(get_most_recent_record)

summarised_data


ods_code,statistic_id,date,total_number_of_records,number_of_document_types,total_size_of_records_in_megabytes
str,str,str,i64,i64,f64
"""M85143""","""675241cd-3615-4459-a927-fc0b4f…","""20240513""",18,1,0.886456
"""ODS_CODE_UNKNOWN""","""a74412b5-0700-4d18-96ca-809b64…","""20240509""",0,1,0.335522
"""H81109""","""eaa28be5-cea0-40de-b668-449f8f…","""20240513""",25,1,1.41418


In [58]:
df = pl.DataFrame(loaded_data.organisation_data).with_columns(cs.by_dtype(pl.datatypes.Decimal).cast(pl.Float64))

sum_weekly_count = cs.matches("daily").sum().name.map(lambda column_name: column_name.replace("daily", "weekly"))
take_average_for_patient_record = pl.col("average_records_per_patient").mean()
get_most_recent_number_of_patients = pl.col("number_of_patients").sort_by('date').last()

df.group_by('ods_code').agg(sum_weekly_count, take_average_for_patient_record, get_most_recent_number_of_patients)


ods_code,weekly_count_stored,weekly_count_viewed,weekly_count_downloaded,weekly_count_deleted,average_records_per_patient,number_of_patients
str,i64,i64,i64,i64,f64,i64
"""H81109""",5,20,6,6,2.851852,9
"""M85143""",0,35,4,1,4.5,4


In [70]:
df = pl.DataFrame(loaded_data.application_data)
count_unique_ids = pl.concat_list('active_user_ids_hashed').flatten().unique().len().alias('Active users count')

df.group_by('ods_code').agg(count_unique_ids)

ods_code,Active users count
str,u32
"""H81109""",5
"""M85143""",1
